# Import Essential Libraries #

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import cv2
import pickle
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.preprocessing.image import img_to_array, load_img 
from sklearn.metrics import roc_curve, auc 
from tensorflow_model_optimization.sparsity import keras as sparsity

#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
#from keras.layers.normalization import BatchNormalization

from datetime import datetime

%load_ext tensorboard
import tensorboard
import tempfile
import zipfile
import os


In [ ]:
adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-4)
bce = tf.keras.losses.BinaryCrossentropy()

def check_binarized_auc(test_dset, model, X_test, y_test):
    '''Input:
            test_dset: test dataset to analyze performance of model on unseen data
            model: keras model trained previously
            
       Output:
            None: print out test AUC in function
    '''  
    y_pred = model.predict(X_test).ravel()
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    auc_model = auc(fpr, tpr)
    output = 'Test AUC: {}'
    print(output.format(auc_model))
    
    #sens and spec at Youden's index
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    sens = tpr[optimal_idx]
    spec = 1 - fpr[optimal_idx]
    print('Sensitivity: ' + str(sens))
    print('Specificity: ' + str(spec))
    
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='Model (area = {:.3f})'.format(auc_model))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    
def loss_plot(hist, epoch=50):
    plt.figure()
    plt.plot(range(epoch), hist.history['loss'], label='Training loss')
    plt.plot(range(epoch), hist.history['val_loss'], label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Value')
    plt.legend()
    # END CODE #
    plt.show()

#functions to get nonzero weights 
def total_nonzero_weights(model):
    '''return total number of nonzero weights (total params)
    includes non-trainable params'''
    weights = model.get_weights()
    count_nz = []
    for i in range(len(weights)):
        count_nz.append(np.count_nonzero(weights[i]))
    return sum(count_nz)

def total_nonzero_trainable_weights(model):
    '''return total number of nonzero trainable weights'''
    w = []
    for v in model.trainable_variables:
        w.append(tf.math.count_nonzero(v))
    return sum(w)    

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("TF Version: ", tf.version.VERSION)

In [ ]:
#ham10000_2k is still available

#pickle.dump([X_train, y_train, X_tune, y_tune, X_test, y_test], open('ham10000_3.9k', 'wb'))
#pickle.dump([iX_train, iy_train, iX_tune, iy_tune, iX_test, iy_test], open('ham10000_3.9k_i', 'wb'))

X_train, y_train, X_tune, y_tune, X_test, y_test = pickle.load(open('ham10000_3.9k', 'rb'))
iX_train, iy_train, iX_tune, iy_tune, iX_test, iy_test = pickle.load(open('ham10000_3.9k_i', 'rb'))

In [ ]:
BATCH_SIZE = 16

train_dset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
tune_dset = tf.data.Dataset.from_tensor_slices((X_tune, y_tune)).batch(BATCH_SIZE)
test_dset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

itrain_dset = tf.data.Dataset.from_tensor_slices((iX_train, iy_train)).batch(BATCH_SIZE)
itune_dset = tf.data.Dataset.from_tensor_slices((iX_tune, iy_tune)).batch(BATCH_SIZE)
itest_dset = tf.data.Dataset.from_tensor_slices((iX_test, iy_test)).batch(BATCH_SIZE)

In [ ]:
#save model
#iv3_b.save('iv3_b.h5')

iv3_b = tf.keras.models.load_model('iv3_b.h5')

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Evaluate Model

In [ ]:
iv3_b.evaluate(test_dset)

Output with sigmoid, default lr, iv3_b: [1.69085515989941, 0.7810499]

# Prune Model

In [ ]:
#Using copied, not-compiled, pre-trained baseline model 
mod = False
iv3_p = False
iv3_f = False

mod = tf.keras.models.clone_model(iv3_b)
mod.set_weights(iv3_b.get_weights()) 

epochs = 5
num_train_samples = X_train.shape[0]

end_step = np.ceil(1.0 * num_train_samples / BATCH_SIZE).astype(np.int32) * epochs
print(end_step)
new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

iv3_p = sparsity.prune_low_magnitude(mod, **new_pruning_params)
iv3_p.summary()

In [ ]:
iv3_p.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

In [ ]:
%tensorboard --logdir={logdir}

In [ ]:
# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

In [ ]:
iv3_p_hist = iv3_p.fit(train_dset, 
                                  epochs=epochs,
                                  verbose = 1,
                                  callbacks=callbacks,
                                  validation_data = tune_dset)

In [ ]:
iv3_f = sparsity.strip_pruning(iv3_p)
iv3_f.summary()

# Evaluate Pruned Model

In [ ]:
#save model
#mn2_pruned.save('MN2_Pruned.h5')
#iv3_f.save('iv3_f.h5')

#iv3_f = tf.keras.models.load_model('iv3_f.h5')


In [ ]:
iv3_p.evaluate(test_dset)

output for iv3_p: [1.826846210932245, 0.77976954]

In [ ]:
#need to compile final model before using it
iv3_f.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])

iv3_f.evaluate(test_dset)

In [ ]:
check_binarized_auc(test_dset, iv3_p, X_test, y_test)
#loss_plot(iv3_p_hist)

In [ ]:
print("model:", total_nonzero_weights(iv3_b),
     "\n pruned:", total_nonzero_weights(iv3_p),
     "\n pruned final:", total_nonzero_weights(iv3_f))